In [1]:
import wandb

In [10]:
!pip install ttach

In [2]:
wandb.login()

True

In [9]:
!python -W ignore ../src/train.py --tags "3#drop#att#" --base "efficientnet_b0" --image_size 224 --lr 1e-3 --epochs 300 \
--batch_size 8 --wandb 1


cuda
Using device: cuda
True
Tesla V100-PCIE-32GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB
wandb!
wandb: Tracking run with wandb version 0.8.18
wandb: Run data is saved locally in ../../shared/kern_segmentation/logs/wandb/run-20191212_171143-iap06m0v
wandb: Syncing run 3#drop#att#_efficientnet_b0_ft_0.001_6
wandb: ⭐️ View project at https://app.wandb.ai/nuvard/kern
wandb: 🚀 View run at https://app.wandb.ai/nuvard/kern/runs/iap06m0v
wandb: Run `wandb off` to turn off syncing.

==> Preparing data
Using cache found in /headless/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
==> Preparing model
Using cache found in /headless/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Adding attention
==> Initialising sampler
=====> Loading indices
=====> Loading samples
=====> Checking distribution
=====> Assigning weights
100%|███████████████████████████████████████| 2244/2244 [07:53<00:00,  4.74it/s]
==> Training model
  0%|                                            

In [54]:
import torch
import sys
from torchsummary import summary

sys.path.append('../src/')
from attention_augmented_conv import AugmentedConv
torch.hub.list('rwightman/gen-efficientnet-pytorch') 
model =  torch.hub.load('rwightman/gen-efficientnet-pytorch','efficientnet_b0', pretrained=True)



model.global_pool = nn.Sequential(
                       AugmentedConv(in_channels=1280, out_channels=200, kernel_size=1, dk=40, dv=4, Nh=1, relative=False, stride=2),
                     #nn.Conv2d(588, 6, kernel_size=1, padding = 1, stride=1, bias=False),
                     nn.BatchNorm2d(200), 
                     #nn.Dropout(p=0.25),
                     nn.AdaptiveAvgPool2d(1)
                     
                     
                      )
        
model.classifier = nn.Sequential(
           
            #nn.BatchNorm1d(6),
            
                     #nn.ReLU(),
            
            #nn.Dropout(p=0.25),
            #
   #nn.BatchNorm1d(1280, eps=1e-05, momentum=0.1),
            #nn.Dropout(p=0.5),
            
            #nn.ReLU(),
            #nn.AdaptiveAvgPool1d(1),
            nn.Linear(200, 6),
            #nn.AdaptiveAvgPool1d(1),
        ) 
temp = model.conv_stem.weight
        #model.conv_stem = AttentionConv2d(in_channels=6, out_channels=64, kernel_size=7, dk=40, dv=4, Nh=4, relative=True, stride=2, padding=3, shape = 24).to(device)
        
model.conv_stem = nn.Conv2d(6, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.conv_stem.weight = nn.Parameter(torch.cat((temp,temp),dim=1))
model.cuda()
summary(model, (6, 240, 240))


conv_out = model(temp_input)#.squeeze()
print(conv_out.shape)

Using cache found in /headless/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Using cache found in /headless/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 122, 122]           1,728
       BatchNorm2d-2         [-1, 32, 122, 122]              64
          SwishJit-3         [-1, 32, 122, 122]               0
            Conv2d-4         [-1, 32, 122, 122]             288
       BatchNorm2d-5         [-1, 32, 122, 122]              64
          SwishJit-6         [-1, 32, 122, 122]               0
 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             264
          SwishJit-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 32, 1, 1]             288
    SqueezeExcite-11         [-1, 32, 122, 122]               0
           Conv2d-12         [-1, 16, 122, 122]             512
      BatchNorm2d-13         [-1, 16, 122, 122]              32
         Identity-14         [-1, 16, 1

RuntimeError: Expected 4-dimensional input for 4-dimensional weight 32 6 3 3, but got 3-dimensional input of size [1280, 32, 32] instead

In [4]:
!python -W ignore ../src/train.py --tags "1#drop0.2" --base "mixnet_s" --image_size 224 --wandb 1 \
          --batch_size 24 --inp_size 1536

cuda
Using device: cuda
True
Tesla V100-PCIE-32GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB
wandb!
wandb: Tracking run with wandb version 0.8.18
wandb: Run data is saved locally in ../../../shared/kern_segmentation/logs/wandb/run-20191207_222740-rqsiwlo7
wandb: Syncing run 1#drop0.2_mixnet_s_ft_0.001_6
wandb: ⭐️ View project at https://app.wandb.ai/nuvard/kern
wandb: 🚀 View run at https://app.wandb.ai/nuvard/kern/runs/rqsiwlo7
wandb: Run `wandb off` to turn off syncing.

==> Preparing data
==> Preparing model
Using cache found in /headless/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Using cache found in /headless/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/mixnet_s-a907afbc.pth" to /headless/.cache/torch/checkpoints/mixnet_s-a907afbc.pth
^C
wandb: Ctrl-c pressed.
wandb: Program failed with code 255. Press ctrl-c to abort syncing.
Traceback (most recent ca

In [ ]:
!python -W ignore ../src/train.py --tags "1#drop0.2#" --base "mnasnet_a1" --image_size 224 --wandb 1